## Setup

### Parameters

In [1]:
import math
from os import urandom
from hashlib import sha3_256, sha3_512, shake_256

from sage.rings.polynomial.polynomial_quotient_ring import PolynomialQuotientRing as polynomial
from sage.matrix.constructor import matrix
from sage.misc.prandom import randrange
from sage.rings.finite_rings.finite_field_constructor import FiniteField 
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
from sage.rings.polynomial.polynomial_quotient_ring import PolynomialQuotientRing as polynomial
from sage.rings.integer import Integer

# Kyber Parameters
q = 3329
q_bytes = 16
k = 2
n = 256


rQ = PolynomialRing(FiniteField(q, 'x'), 'x', sparse=True)
x = rQ.gen()
f = x^n + 1
RQ = rQ.quotient(f)


DEBUG = True

### Helper Functions

In [2]:
def RandomList(length, cbd=False):
    out = [randrange(q) for i in range(length)]
    if cbd:
        out = FauxCbd(out)
    return out

def FauxCbd(r: list):
    out = []
    for i in r:
        out.append((i % 5) - 2) # Restrict to -2 <= n <= 2
    return out

def RandPolyUniform(length):
    return RQ(RandomList(length))

def RandPolyCbd(length) -> polynomial:
    return RQ(RandomList(length, cbd=True))

def RandListCbd(length) -> list:
    return RandomList(length, cbd=True)

def BytesNeed4Bits(bits: int) -> int:
    return ((bits+7) & (-8))//8

def RandInt(bits: int) -> int:
    m = Integer(int.from_bytes(urandom(BytesNeed4Bits(bits)), 'big'))
    m &= 2**n-1
    return m

def Poly2Bytes(poly: polynomial) -> bytes:
    out = b''
    p = poly.coefficients()[0].list()
    for c in p:
        c = int(c)
        cb = c.to_bytes(q_bytes, 'big')
        out += cb
    return out

def Bytes2ListBit(b: bytes) -> list:
    out = []
    for byte in b:
        for i in range(0,8):
            bit = (byte >> i) & 1
            out.append(bit)
    return out

def Compress(poly: polynomial):
    q2 = math.ceil(q/2)
    return poly * q2

def Decompress(poly: polynomial) -> int:
    return [(1 if 3*(q/4) > Integer(i) > q/4 else 0) for i in poly]

def dbg(label: str, *args: str):
    if DEBUG:
        s = f'{label}:\n'
        for arg in args:
            s += f'{arg}\n'
        if len(args) == 0:
            s = s[:-2]
        print(s)

## NTT Multiplication

In [3]:
#NTT PARAMETERS FOR KYBER
mont_r         = 2285  # 2^16 % q
mont_r2        = 1353  # 2^32 % q
mont_r_inv     = 169   # (1 / 2^16) % q
mont_mask      = 65535 # 2^16 - 1,
q_inv          = 3327  # -1 / 3329 ^ 2^16,
root_of_unity  = 17
f              = 1441  # 2^32 / 128 % q
# zetas      = [(mont_r * pow(root_of_unity,  br(i,7), q)) % q for i in range(128)],
zetas          = [2285, 2571, 2970, 1812, 1493, 1422, 287, 202, 3158, 622, 1577, 182, 962, 2127, 1855, 1468, 
                  573, 2004, 264, 383, 2500, 1458, 1727, 3199, 2648, 1017, 732, 608, 1787, 411, 3124, 1758, 
                  1223, 652, 2777, 1015, 2036, 1491, 3047, 1785, 516, 3321, 3009, 2663, 1711, 2167, 126, 1469, 
                  2476, 3239, 3058, 830, 107, 1908, 3082, 2378, 2931, 961, 1821, 2604, 448, 2264, 677, 2054, 
                  2226, 430, 555, 843, 2078, 871, 1550, 105, 422, 587, 177, 3094, 3038, 2869, 1574, 1653, 3083, 
                  778, 1159, 3182, 2552, 1483, 2727, 1119, 1739, 644, 2457, 349, 418, 329, 3173, 3254, 817, 
                  1097, 603, 610, 1322, 2044, 1864, 384, 2114, 3193, 1218, 1994, 2455, 220, 2142, 1670, 2144, 
                  1799, 2051, 794, 1819, 2475, 2459, 478, 3221, 3021, 996, 991, 958, 1869, 1522, 1628]
# zetas      = [(pow(root_of_unity,  br(i,7), q)) % q for i in range(128)]

def montgomery_reduce(a):
    """
    This is not proper mont. reduction.
    But this is faster than the normal impl
    because python is weird.
    
    Proper impl is commented out at the bot.
    of the file...
    
    a -> R^(-1) a mod q
    """
    return a * mont_r_inv % q

def mont_mul(a, b):
    """
    Multiplication then Montgomery reduction
    
    Ra * Rb -> Rab
    """
    c = a * b
    return montgomery_reduce(c)

def ntt_base_multiplication(a0, a1, b0, b1, zeta):
    r0  = mont_mul(a1, b1)
    r0  = mont_mul(r0, zeta)
    r0 += mont_mul(a0, b0)
    r1  = mont_mul(a0, b1)
    r1 += mont_mul(a1, b0)
    return r0, r1
    
def ntt_coefficient_multiplication(f_coeffs, g_coeffs):
    new_coeffs = []
    for i in range(64):
        r0, r1 = ntt_base_multiplication(
                            f_coeffs[4*i+0], f_coeffs[4*i+1],
                            g_coeffs[4*i+0], g_coeffs[4*i+1],
                            zetas[64+i])
        r2, r3 = ntt_base_multiplication(
                            f_coeffs[4*i+2], f_coeffs[4*i+3],
                            g_coeffs[4*i+2], g_coeffs[4*i+3],
                            -zetas[64+i])
        new_coeffs += [r0, r1, r2, r3]
    return RQ(new_coeffs)

def br(i, k):
    """
    bit reversal of an unsigned k-bit integer
    """
    bin_i = bin(i & (2**k - 1))[2:].zfill(k)
    return int(bin_i[::-1], 2)

def to_ntt(a):
    """
    Convert a polynomial to number-theoretic transform (NTT) form in place
    The input is in standard order, the output is in bit-reversed order.
    NTT_ZETAS also has the Montgomery factor 2^16 included, so NTT 
    additionally maps to Montgomery domain.
    
    Only implemented (currently) for n = 256
    """
    k, l = 1, 128
    coeffs = a.list()
    print(coeffs)
    print()
    while l >= 2:
        start = 0
        while start < 256:
            zeta = zetas[k]
            k = k + 1
            for j in range(start, start + l):
                t = mont_mul(zeta, coeffs[j+l])
                coeffs[j+l] = coeffs[j] - t
                coeffs[j]   = coeffs[j] + t
            start = l + (j + 1)
        l = l >> 1
    print(coeffs)
    print()
    return RQ(coeffs)
    
def from_ntt(a):
    """
    Convert a polynomial from number-theoretic transform (NTT) form in place
    and multiplication by Montgomery factor 2^16.
    The input is in bit-reversed order, the output is in standard order.
    
    Because of the montgomery multiplication, we have:
        f != f.to_ntt().from_ntt()
        f = (1/2^16) * f.to_ntt().from_ntt()
    
    To recover f we do
        f == f.to_ntt().from_ntt().from_montgomery()
        
    Only implemented (currently) for n = 256
    """     
    l, l_upper = 2, 128
    k = l_upper - 1
    coeffs = a.list()
    print(coeffs)
    while l <= 128:
        start = 0
        while start < 256:
            zeta = zetas[k]
            k = k - 1
            for j in range(start, start+l):
                t = coeffs[j]
                coeffs[j]   = t + coeffs[j+l]
                coeffs[j+l] = coeffs[j+l] - t
                coeffs[j+l] = mont_mul(zeta, coeffs[j+l])
            start = j + l + 1
        l = l << 1
    for j in range(256):
        coeffs[j] = mont_mul(coeffs[j], f)
        
    return RQ(coeffs)

def polymul_ntt_ct_gs(a, b):
    """Compute a polynomial multiplication by computing iNTT(NTT(a) o NTT(b)).

    Works for both the cyclic and the negacyclic case (with the correct twiddles).

    Parameters
    ----------
    a : Poly
        first multiplicand polynomial with n coefficients.
    b : Poly
        second multiplicand polynomial with n coefficients.
    twiddlesNtt : list
        twiddles for the foward NTT as computed by `precomp_ct_cyclic` or `precomp_ct_negacyclic`.
    tiwddlesInvntt : list
        twiddles for the inverse nTT as computed by `precomp_gs_cyclic` or `precmp_gs_negacyclic`.
    Returns
    ----------
    Poly
        product a*b with n coefficients.
    """
    antt = to_ntt(a)
    bntt = to_ntt(b)
    alist = antt.list()
    blist = bntt.list()
    cntt = ntt_coefficient_multiplication(alist,blist)
    return from_ntt(cntt)
    
def nttmul(A, B):
    """
    Denoted A @ B
    """
    new_elements = [[sum(polymul_ntt_ct_gs(a, b) for a,b in zip(A_row, B_col)) for B_col in B.transpose().rows()] for A_row in A.rows()]
    return matrix(new_elements)

## INDCPA Public Key Encryption (K-PKE)

### K-PKE KeyGen

In [8]:
def KPKE_Keygen(NTT=False) -> (matrix, matrix, matrix):
    dbg('===== kpke_keygen =====')

    # Initialize
    A = [[[None] for _ in range(0, k)] for _ in range(0, k)]
    s = [[None] for _ in range(0, k)]
    e = [[None] for _ in range(0, k)]

    # A is a k*k dimension matrix of polynomials with n terms
    for i in range(0, k):
        for j in range(0, k):
            A[i][j] = RandPolyUniform(n)
    A = matrix(A)

    dbg('A', A)

    # s is a k*1 dimension matrix of polynomials with n terms
    for i in range(0, k):
        s[i] = [RandPolyCbd(n)]
    s = matrix(s)

    dbg('s', s)

    # e is a k*1 dimension matrix of polynomials with n terms
    for i in range(0, k):
        e[i] = [RandPolyCbd(n)]
    e = matrix(e)

    dbg('e', e)

#   Compute t = A*s*e:
#   A*s is a k * 1 matrix of polynomials with n terms
#   A*s+e is a k * 1 matrix polynomials with n terms
#   t is a k*1 dimension matrix
#
#   Example when k=2:
#   |     A     |   |  s  |   |  e  |
#   | :-- | :-- |   | :-- |   | :-- |
#   | 0,0 | 0,1 |   |  0  |   |  0  |
#   | 1,0 | 1,1 |   |  1  |   |  1  |
#
#   |             A * s             |
#   | :---------------------------- |
#   | A[0,0] * s[0] + A[0,1] * s[1] |
#   | A[1,0] * s[0] + A[1,1] * s[1] |
#
#   |     As+e     |
#   | :----------- |
#   | As[0] + e[0] |
#   | As[1] + e[1] |

    if NTT:
        t = nttmul(A,s)
        t = t + e
    else:
        t = A * s + e

    dbg('t', t, '\n')

    return (A, t, s)

### K-PKE Encrypt

In [5]:
def KPKE_Encrypt(A: matrix, t: matrix, m: int, r: polynomial, NTT=False) -> (polynomial, polynomial):
    dbg('===== kpke_encrypt =====')

    # Initialize
    rr = [[None] for _ in range(0, k)]
    e1 = [[None] for _ in range(0, k)]
    e2 = [None] * n
    
    # Ensure that m does not have more bits than n bits
    if len(m.bits()) > n:
        raise ValueError('m has more bits than n!')
    mb = m.bits()

    dbg('Bits of m', mb)

    # N is nonce used to deterministicly modify r
    N = 0

    # We need m to be at least n bits long.
    # Pad mm with 0s until desired length is reached
    pad = [0 for _ in range(0, n - len(mb))]
    mbp = RQ(mb + pad)

    dbg('Polynomial m', mbp)

    # Compress m
    mbpc = Compress(mbp)

    dbg('Compressed m', mbpc)

    # Generate r, e1, e2
    # r is a k*1 matrix of polynomials with n terms
    for i in range(0, k):
        tpoly = [None] * n
        for j in range(0, n):
            tpoly[j] = r[j] + N
        tpoly = FauxCbd(tpoly)
        tpoly = RQ(tpoly)
        rr[i] = [tpoly]
        N += 1
    rr = matrix(rr)
    
    dbg('rr', rr)

    # e1 is a k*1 matrix of polynomials with n terms
    for i in range(0, k):
        tpoly = [None] * n
        for j in range(0, n):
            tpoly[j] = r[j] + N
        tpoly = FauxCbd(tpoly)
        tpoly = RQ(tpoly)
        e1[i] = [tpoly]
        N += 1
    e1 = matrix(e1)

    dbg('e1', e1)

    # e2 is an n-length polynomial with n terms
    for i in range(0, n):
        e2[i] = r[i] + N
    e2 = FauxCbd(e2)
    e2 = RQ(e2)

    dbg('e2', e2)
    
    if NTT:
        u = nttmul(A.transpose(),rr)
        u = u + e1
        v = nttmul(t.transpose(),rr)
        v = v + e2 + mbpc
    else:
        u = A.transpose() * rr + e1
        v = t.transpose() * rr + e2 + mbpc

    dbg('u', u)
    dbg('v', v, '\n')

    return (u, v)

    

### K-PKE Decrypt

In [6]:
def KPKE_Decrypt(u: matrix, v: matrix, s: matrix, NTT=False) -> int:
    dbg('===== kpke_decrypt =====')
    
    # Compute a noisy result mn
    if NTT:
        mn = nttmul(s.transpose(),u)
        mn = v - mn
    else:
        mn = v - s.transpose() * u

    mn = mn.coefficients()[0]

    dbg('Noisy recovered m', mn)
   
    mn_c = mn.list()
    mn_c.reverse()

    # Decompress and remove the noise
    m_rec = Decompress(mn_c)

    dbg('Decompressed m', list(reversed(m_rec)), '\n')

    # Convert to integer
    m_rec = int(''.join([str(x) for x in m_rec]), 2)

    return m_rec

### K-PKE Test

In [7]:
# Alice generates a public key (A, t),
# and a private key s
A, t, s = KPKE_Keygen()

# Alice sends Bob her pk
# Bob chooses a random message m
# and encrypts it using Alice's pk
# and some randomness r to produce the ciphertext (u, v)
m = RandInt(n)
r = RandListCbd(n)
u, v = KPKE_Encrypt(A, t, m, r)

# Bob sends Alice (u, v).
# Alice can then recover the message m
mr = KPKE_Decrypt(u, v, s)

dbg('Alice\'s m', mr)
dbg('Bob\'s m', m)
dbg(m.bits())
if m != mr:
    raise ValueError('Alice and Bob\'s messages do not match, final decompression likely failed. Try increasing the value of the prime q')


===== kpke_keygen =====
A:
[   2090*xbar^255 + 74*xbar^254 + 220*xbar^253 + 2875*xbar^252 + 1436*xbar^251 + 929*xbar^250 + 285*xbar^249 + 336*xbar^248 + 1052*xbar^247 + 2114*xbar^246 + 1899*xbar^245 + 425*xbar^244 + 2257*xbar^243 + 1158*xbar^242 + 3069*xbar^241 + 460*xbar^240 + 185*xbar^239 + 93*xbar^238 + 186*xbar^237 + 2915*xbar^236 + 2004*xbar^235 + 2461*xbar^234 + 2134*xbar^233 + 2906*xbar^232 + 776*xbar^231 + 3294*xbar^230 + 3183*xbar^229 + 721*xbar^228 + 2242*xbar^227 + 536*xbar^226 + 254*xbar^225 + 3182*xbar^224 + 521*xbar^223 + 2335*xbar^222 + 471*xbar^221 + 716*xbar^220 + 1337*xbar^219 + 3171*xbar^218 + 3226*xbar^217 + 1802*xbar^216 + 3326*xbar^215 + 2088*xbar^214 + 1179*xbar^213 + 3005*xbar^212 + 169*xbar^211 + 3270*xbar^210 + 1796*xbar^209 + 1454*xbar^208 + 456*xbar^207 + 3000*xbar^206 + 1902*xbar^205 + 179*xbar^204 + 1392*xbar^203 + 2402*xbar^202 + 2409*xbar^201 + 2118*xbar^200 + 2209*xbar^199 + 2374*xbar^198 + 2050*xbar^197 + 1061*xbar^196 + 1485*xbar^195 + 70*xbar^194 + 3

# INDCCA Key Exchange Mechanism (ML-KEM)

## ML-KEM KeyGen

In [9]:
def MLKEM_KeyGen() -> ((polynomial, polynomial), (polynomial, polynomial, int)):
    dbg('===== MLKEM_KeyGen =====')
    
    z = RandInt(n)
    A, t, s = KPKE_Keygen()
    ek = (A, t)
    Ht = sha3_256(Poly2Bytes(t)).digest()

    dbg('SHA3-256(t)', Ht.hex(), '\n')

    dk = (s, ek, Ht, z)

    return ek, dk

## ML-KEM Encaps

In [10]:
def MLKEM_Encaps(ek: (polynomial, polynomial)) -> (bytes, bytes):
    dbg('===== MLKEM_Encaps =====')

    m = RandInt(n)
    dbg('m', m)

    A, t = ek
    Ht = sha3_256(Poly2Bytes(t)).digest()

    dbg('SHA3-256(t)', Ht.hex())

    Kr = sha3_512(int(m).to_bytes(BytesNeed4Bits(n), 'big') + Ht).digest()
    K, r = (Kr[:BytesNeed4Bits(n)], Kr[BytesNeed4Bits(n):])
    r = FauxCbd(Bytes2ListBit(r))

    dbg('r', r, '\n')
    
    c = KPKE_Encrypt(A, t, m, r)

    return K, c

## ML-KEM Decaps

In [11]:
def MLKEM_Decaps(c: (polynomial, polynomial), dk: (polynomial, (polynomial, polynomial), bytes, int)) -> bytes:
    dbg('===== MLKEM_Decaps =====')
    s, ek, h, _ = dk
    A, t = ek
    u, v = c

    mprime = KPKE_Decrypt(u, v, s)

    dbg('m\'', mprime)

    Krprime = sha3_512(int(mprime).to_bytes(BytesNeed4Bits(n), 'big') + h).digest()
    Kprime, rprime = (Krprime[:BytesNeed4Bits(n)], Krprime[BytesNeed4Bits(n):])
    rprime = FauxCbd(Bytes2ListBit(rprime))

    dbg('r\'', rprime)

    uprime, vprime = KPKE_Encrypt(A, t, Integer(mprime), rprime)

    dbg('u\'', uprime)
    dbg('v\'', vprime, '\n')

    return Kprime



## ML-KEM Test

In [71]:
# Alice runs KeyGen()
pkA, skA = MLKEM_KeyGen()

# Bob receives pkA from Alice,
# then runs Encaps() to generate
# his copy of the shared secret ssB, 
# and a ciphertext c
ssB, c = MLKEM_Encaps(pkA)

# Bob sends c to Alice,
# who then uses her secret key
# to generate her copy of
# the shared secret ssA
ssA = MLKEM_Decaps(c, skA)

dbg('Bob\'s Shared Secret', ssB.hex())
dbg('Alice\'s Shared Secret', ssA.hex())
dbg('ssA == ssB?', ssA == ssB)
if(ssA != ssB):
    raise ValueError('The shared keys do not match!')

===== MLKEM_KeyGen =====
===== kpke_keygen =====
A:
[                 2902*xbar^255 + 1236*xbar^254 + 2401*xbar^253 + 3003*xbar^252 + 3301*xbar^251 + 2799*xbar^250 + 234*xbar^249 + 1299*xbar^248 + 1149*xbar^247 + 1410*xbar^246 + 336*xbar^245 + 925*xbar^244 + 607*xbar^243 + 227*xbar^242 + 1930*xbar^241 + 1465*xbar^240 + 2558*xbar^239 + 1545*xbar^238 + 2836*xbar^237 + 1189*xbar^236 + 489*xbar^235 + 2986*xbar^234 + 2806*xbar^233 + 1768*xbar^232 + 2392*xbar^231 + 3209*xbar^230 + 3243*xbar^229 + 591*xbar^228 + 2330*xbar^227 + 115*xbar^226 + 2284*xbar^225 + 2498*xbar^224 + 1084*xbar^223 + 273*xbar^222 + 1383*xbar^221 + 2553*xbar^220 + 821*xbar^219 + 1751*xbar^218 + 1258*xbar^217 + 39*xbar^216 + 2512*xbar^215 + 2698*xbar^214 + 2703*xbar^213 + 2845*xbar^212 + 280*xbar^211 + 1380*xbar^210 + 383*xbar^209 + 2903*xbar^208 + 2105*xbar^207 + 1199*xbar^206 + 3090*xbar^205 + 424*xbar^204 + 3227*xbar^203 + 418*xbar^202 + 3170*xbar^201 + 2466*xbar^200 + 318*xbar^199 + 1713*xbar^198 + 1750*xbar^197 + 134

# Key Exchange Tests

## Init

In [92]:
DEBUG = False

# Alice and Bob create independent *static* (pk, sk) pairs
pkA, skA = MLKEM_KeyGen()
pkB, skB = MLKEM_KeyGen()

## Unilaterally Authenticated Key Exchange (UAKE)

In [93]:
# Alice generates a new set of temporary keys
tpkA, tskA = MLKEM_KeyGen()
# Alice encapsulates Bob's static public key (pkB)
# to generare a ciphertext and shared secret (tcA, tssA) to send to Bob
tssA, cA = MLKEM_Encaps(pkB)

# Alice sends (tpkA, cA) to Bob
# Bob encapsulates Alice's temporary public key (tpkA)
# to generare a ciphertext and shared secret (cB, tssB)
# to send to Alice
tssB, cB = MLKEM_Encaps(tpkA)
# Bob decapsulates Alice's ciphertext to produce tssAprime
tssAprime = MLKEM_Decaps(cA, skB)
# Bob hashes tssB and tssAprime to create his copy of the final shared key (ssB)
ssB = shake_256(tssB + tssAprime).digest(32)

# Bob sends Alice his ciphertext tcB
# Alice decapsulates Bob's tcB using her temporary secret key
# to recover Bob's temporary shared secret
tssBprime = MLKEM_Decaps(cB, tskA)
# Alice hashes tssBprime and tssA to produce her copy of the final shared key (ssA)
ssA = shake_256(tssBprime + tssA).digest(32)

print('Alice\'s SS:')
print(ssA.hex())
print()
print('Bob\'s SS:')
print(ssB.hex())
print()
print('Length of shared secret:', len(ssA), 'bytes')
print()
print('ssA == ssB?', ssA == ssB)

Alice's SS:
cb6638655ba084af69a820a10399ef33748337df66e54c0ce222c698d9c6004d

Bob's SS:
cb6638655ba084af69a820a10399ef33748337df66e54c0ce222c698d9c6004d

Length of shared secret: 32 bytes

ssA == ssB? True


## Mutually Authenticated Key Exchange (AKE)

In [94]:
# Alice generates a new set of temporary keys
tpkA, tskA = MLKEM_KeyGen()
# Alice encapsulates Bob's static public key (pkB)
# to generare a ciphertext and shared secret (tcA, tSSA) to send to Bob
tssA, cA = MLKEM_Encaps(pkB)

# Alice sends (tpkA, cA) to Bob
# Bob encapsulates Alice's temporary public key (tpkA)
# to generare a ciphertext and shared secret (tcB, tssB)
tssB, cB = MLKEM_Encaps(tpkA)
# Bob encapsulates Alice's static public key (pkA)
# to generare a second ciphertext and shared secret (tcB2, tssB2)
tssB2, cB2 = MLKEM_Encaps(pkA)
# Bob decapsulates Alice's tcA using his static secret key (skB)
# to recover Alice's temporary shared secret
tssAprime = MLKEM_Decaps(cA, skB)
# Bob hashes tssB2, tssB2, and tssAprime to create his copy of the final shared key (ssB)
ssB = shake_256(tssB + tssB2 + tssAprime).digest(32)

# Bob sends tcB, tcB2 to Alice
# Alice decapsulates Bob's cB using her temprary secret key (tskA)
tssBprime = MLKEM_Decaps(cB, tskA)
# Alice decapsulates Bob's cB2 using her static secret key (skA)
tssB2prime = MLKEM_Decaps(cB2, skA)
# Alice hashes tssBprime, tssB2prime, and tssA to create her copy of the final shared key (ssA)
ssA = shake_256(tssBprime + tssB2prime + tssA).digest(32)

print('Alice\'s SS:')
print(ssA.hex())
print()
print('Bob\'s SS:')
print(ssB.hex())
print()
print('Length of shared secret:', len(ssA), 'bytes')
print()
print('ssA == ssB?', ssA == ssB)


Alice's SS:
7e01cad2813afe05f3c4dc0f542986bb3bcc131953674be7d82dc303d17cac42

Bob's SS:
7e01cad2813afe05f3c4dc0f542986bb3bcc131953674be7d82dc303d17cac42

Length of shared secret: 32 bytes

ssA == ssB? True
